In [12]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive


/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Change directory in google drive
%cd '//content//drive//My Drive/Colab Notebooks//'

/content/drive/My Drive/Colab Notebooks


In [14]:
#import pacjages
import os
import time
import json
import pandas as pd
import tweepy
import datetime

In [24]:
# this was taken from https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25, it uses a class to create the api and define the things we want to extract. Right now I have used my 
#API credentials but we need to change this using Rob method
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    '''
    ##################################Bit I am changing
    api_keys_fp = '//content//drive//My Drive//Colab Notebooks//twitter_credentials.json'
    def load_api(keys_file):
      # Load twitter credentials
      with open(keys_file, 'r') as file:
        creds = json.load(file)

    # Use credentials to set up API access authorisation
    #auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    #auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
    #api = tweepy.API(auth, wait_on_rate_limit=True)
    
    #return api
    twitter_keys = {
        'consumer_key':        creds['CONSUMER_KEY'],
        'consumer_secret':     creds['CONSUMER_SECRET'],
        'access_token_key':    creds['ACCESS_TOKEN'],
        'access_token_secret': creds['ACCESS_SECRET']
    }
    '''
###############################################

    twitter_keys = {
        'consumer_key':        'iwvyIvriy0AwHR0T7n1AfJJDi',
        'consumer_secret':     '8A3UbQzrOmLKq9hhTwghp0oH4uga0p84GDNNiqeCR7gAhc0JPH',
        'access_token_key':    '1291308136679317504-opzlc5uR8yPUOYf5xtB07KVaOFKE7U',
        'access_token_secret': 'n0fQkOk7atzzYksXXtiyVcN9B8Wc3LukVhdA2wKt9rsDH'
    }
 
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="dril", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True)        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:

                mined = {

                    #user info
                    'user_id':         item.user.id,
                    'name':            item.user.name,
                    'screen_name':     item.user.screen_name,
                    'location':        item.user.location,
                    'user_description':item.user.description,
                    'user_friends_n':  item.user.friends_count,
                    'user_followers_n':item.user.followers_count,
                    'listed_count':    item.user.listed_count,
                    'prof_created_at': item.user.created_at,
                    'favourites_count':item.user.favourites_count,
                    'verified':        item.user.verified,
                    'statuses_count':  item.user.statuses_count,
                    'contributors_enabled': item.user.contributors_enabled,

                    #tweets data
                    'tweet_created_at':item.created_at,
                    'tweet_id':        item.id,
                    'text':            item.full_text,
                    'truncated':       item.truncated,
                    'display_text_range': item.display_text_range,
                    #'tweet_entities':  item.entities,
                    'source':          item.source,
                    'source_url':      item.source_url,
                    'in_reply_to_status_id':item.in_reply_to_status_id,
                    'in_reply_to_user_id': item.in_reply_to_user_id,
                    'in_reply_to_screen_name': item.in_reply_to_screen_name,
                    'is_quote_status': item.is_quote_status,
                    'retweet_count':   item.retweet_count,                    
                    'favourite_count': item.favorite_count,
                    'hashtags':        item.entities['hashtags'],
                    'mentions':        item.entities['user_mentions'],
                    #'location':        item.place,
                    'source_device':   item.source,
                    'retweeted':       item.retweeted
                    
                      
                }
                
                try:
                    mined['retweet_text'] = item.retweeted_status.full_text
                except:
                    mined['retweet_text'] = 'None'
                try:
                    mined['quote_text'] = item.quoted_status.full_text
                    mined['quote_screen_name'] = status.quoted_status.user.screen_name
                except:
                    mined['quote_text'] = 'None'
                    mined['quote_screen_name'] = 'None'
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data

In [45]:
#now use the scraping function to get a list of journalist twitter handles based on a keyword
%run LL_function_to_scrape_journalists_webpage.ipynb
twitter_handles = scrape_journalist_website('cyber')
users = twitter_handles['twitter_handle']

In [47]:
list(users)

['jennystrasburg',
 'dannsimmons',
 'LeoKelion',
 'gordoncorera',
 'joetidy',
 '_lucyingham',
 'dannyjpalmer',
 'SophiaFurber',
 'SCFGallagher',
 'MsHannahMurphy',
 'JesscaHaworth',
 'Ad_Nauseum74']

In [40]:
#initialize the class
miner=TweetMiner(result_limit = 200)

In [ ]:
#mined_tweets = miner.mine_user_tweets(user=USER, max_pages=17)

#mined_tweets_df= pd.DataFrame(mined_tweets)

In [41]:
# this is taken from the same website, it basically run the scraping on a list of users
# there is a part where it pauses of it gets some errors (probably due to API tweets limits)
#probably that part can be modified or tweaked to work better.
import time

handle_list= list(users)

twitter_dict={}
counter=0

for name in handle_list:
    try:
      twitter_dict[name]=[]
      twitter_dict[name].append(miner.mine_user_tweets(user=name, max_pages=17))
      counter = counter +1

      #if counter%40==0:
        #time.sleep(900) #15 minute sleep time
    #if name invalid print name and remove key
    except:
      print(name, 'is invalid or locked')
      twitter_dict.pop(name)
    
    
all_tweets=pd.concat([pd.DataFrame(twitter_dict[i][0]) for i in twitter_dict])

MrNiallMcGarry is invalid or locked
AndyRoocroft is invalid or locked


In [ ]:
#check the dataframe
min(all_tweets['created_at'])

Timestamp('2009-02-23 14:11:02')

In [42]:
all_tweets.head()

,user_id,name,screen_name,location,user_description,user_friends_n,user_followers_n,listed_count,prof_created_at,favourites_count,verified,statuses_count,contributors_enabled,tweet_created_at,tweet_id,text,truncated,display_text_range,source,source_url,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,retweet_count,favourite_count,hashtags,mentions,source_device,retweeted,retweet_text,quote_text,quote_screen_name
0,9947892,Paul Cockerton,paulcockerton,East London via East Hull,Work: @DailyMirror. paul.cockerton@mirror.co.u...,2460,3644,112,2007-11-04 21:16:24,5197,True,6933,False,2020-08-10 17:52:11,1292881442549182466,@gemmaoaten Always tempting to tell the whole ...,False,"[12, 119]",Twitter for iPhone,http://twitter.com/download/iphone,1.2928e+18,3.37364e+08,gemmaoaten,False,0,1,[],"[{'screen_name': 'gemmaoaten', 'name': 'Gemma ...",Twitter for iPhone,False,None,None,None
1,9947892,Paul Cockerton,paulcockerton,East London via East Hull,Work: @DailyMirror. paul.cockerton@mirror.co.u...,2460,3644,112,2007-11-04 21:16:24,5197,True,6933,False,2020-08-10 17:37:45,1292877812123279362,@CockertonFc good luck this season.,False,"[0, 35]",Twitter for iPhone,http://twitter.com/download/iphone,NaN,1.28745e+18,CockertonFc,False,5,9,[],"[{'screen_name': 'CockertonFc', 'name': 'Cocke...",Twitter for iPhone,False,None,None,None
2,9947892,Paul Cockerton,paulcockerton,East London via East Hull,Work: @DailyMirror. paul.cockerton@mirror.co.u...,2460,3644,112,2007-11-04 21:16:24,5197,True,6933,False,2020-08-10 15:41:41,1292848603824820224,RT @RossMcCaff: Mad how we're about 3 days awa...,False,"[0, 106]",TweetDeck,https://about.twitter.com/products/tweetdeck,NaN,NaN,None,False,37,0,[],"[{'screen_name': 'RossMcCaff', 'name': 'Ross M...",TweetDeck,False,Mad how we're about 3 days away from this bein...,None,None
3,9947892,Paul Cockerton,paulcockerton,East London via East Hull,Work: @DailyMirror. paul.cockerton@mirror.co.u...,2460,3644,112,2007-11-04 21:16:24,5197,True,6933,False,2020-08-07 12:58:55,1291720477485010945,@Tigerfan11 Do you have more info on the campa...,False,"[12, 56]",Twitter for iPhone,http://twitter.com/download/iphone,1.29171e+18,1.98564e+07,Tigerfan11,False,0,0,[],"[{'screen_name': 'Tigerfan11', 'name': 'Paul R...",Twitter for iPhone,False,None,None,None
4,9947892,Paul Cockerton,paulcockerton,East London via East Hull,Work: @DailyMirror. paul.cockerton@mirror.co.u...,2460,3644,112,2007-11-04 21:16:24,5197,True,6933,False,2020-08-06 18:10:10,1291436419848773633,A nurse and her Tory voting husband claim they...,False,"[0, 137]",TweetDeck,https://about.twitter.com/products/tweetdeck,NaN,NaN,None,True,0,0,[],[],TweetDeck,False,None,None,None


In [28]:
len(all_tweets)

27842

In [44]:
# save CSV file
all_tweets.to_csv(r'//content//drive//My Drive/Colab Notebooks//politics_tweets_12082020.csv',index = False)